# Manual catalog verification

This notebook presents methods for verifying that a directory contains a valid HATS catalog and performing manual verification through inspecting the catalog metadata and contents.

## 1. Directory verification

The HATS library provides a method to verify that a directory contains the appropriate metadata files.

There are a few flavors of the validation, and the quickest one doesn't take any additional flags:

In [1]:
from hats.io.validation import is_valid_catalog
import hats
from upath import UPath

gaia_catalog_path = UPath("https://data.lsdb.io/hats/gaia_dr3/gaia/")
is_valid_catalog(gaia_catalog_path)

True

### 1.1. Explaining the input and output

The `strict` argument takes us through a different code path that rigorously tests the contents of all ancillary metadata files and the consistency of the partition pixels.

Here, we use the `verbose=True` argument to print out a little bit more information about our catalog. It will repeat the path that we're looking at, display the total number of partitions, and calculate the approximate sky coverage, based on the area of the HATS tiles.

The `fail_fast` argument will determine if we break out of the method at the first sign of trouble or keep looking for validation problems. This can be useful if you're debugging multiple points of failure in a catalog.

In [2]:
is_valid_catalog(gaia_catalog_path, verbose=True, fail_fast=False, strict=True)

Validating catalog at path https://data.lsdb.io/hats/gaia_dr3/gaia/ ... 
Found 2016 partitions.
Approximate coverage is 100.00 % of the sky.


True

## 2. Columns and data types

HATS tables are backed by parquet files. These files store metadata about their columns, the data types, and even the range of values.

The columns and types are stored on the `catalog.schema` attribute with a `pyarrow.Schema` object. You can find more details on this object and its use [in the pyarrow documents](https://arrow.apache.org/docs/python/generated/pyarrow.Schema.html)

Gaia has a lot of columns, so this display is long!

In [3]:
catalog_object = hats.read_hats(gaia_catalog_path)
catalog_object.schema

_healpix_29: int64
solution_id: int64
designation: string
source_id: int64
random_index: int64
ref_epoch: double
ra: double
ra_error: float
dec: double
dec_error: float
parallax: double
parallax_error: float
parallax_over_error: float
pm: float
pmra: double
pmra_error: float
pmdec: double
pmdec_error: float
ra_dec_corr: float
ra_parallax_corr: float
ra_pmra_corr: float
ra_pmdec_corr: float
dec_parallax_corr: float
dec_pmra_corr: float
dec_pmdec_corr: float
parallax_pmra_corr: float
parallax_pmdec_corr: float
pmra_pmdec_corr: float
astrometric_n_obs_al: int16
astrometric_n_obs_ac: int16
astrometric_n_good_obs_al: int16
astrometric_n_bad_obs_al: int16
astrometric_gof_al: float
astrometric_chi2_al: float
astrometric_excess_noise: float
astrometric_excess_noise_sig: float
astrometric_params_solved: int8
astrometric_primary_flag: bool
nu_eff_used_in_astrometry: float
pseudocolour: float
pseudocolour_error: float
ra_pseudocolour_corr: float
dec_pseudocolour_corr: float
parallax_pseudocolour_

### 2.1. Column statistics

Parquet maintains basic statistics about the data inside its files. This includes the minimum value, maximum value, and the number of null (None, or unspecified) rows for that column.

We provide a method that consumes all of the min, max, and null counts, and provides global values of min and max, and a total sum of the null counts.

In [4]:
catalog_object.aggregate_column_statistics()

,min_value,max_value,null_count,row_count
column_names,,,,
solution_id,1636148068921376768,1636148068921376768,0,1812731847
designation,Gaia DR3 1000000057322000000,Gaia DR3 999999988604363776,0,1812731847
...,...,...,...,...
ebpminrp_gspphot_upper,9.999999747378752e-05,4.226200103759766,1341620712,1812731847
libname_gspphot,A,PHOENIX,1341620712,1812731847


Again, gaia has a lot of columns. To make the most of this output, you can either use a pandas option to display all of the rows:

```
import pandas as pd
pd.set_option('display.max_rows', None)
```

Or restrict the columns to those you care about with a keyword argument:

In [5]:
catalog_object.aggregate_column_statistics(include_columns=["ra", "dec", "ref_epoch"])

,min_value,max_value,null_count,row_count
column_names,,,,
ref_epoch,2.016000e+03,2016.000000,0,1812731847
ra,3.409624e-07,360.000000,0,1812731847
dec,-8.999288e+01,89.990052,0,1812731847


## About

**Authors**: Melissa DeLucchi

**Last updated on**: Oct 27, 2025

If you use `lsdb` for published research, please cite following [instructions](https://docs.lsdb.io/en/stable/citation.html).